In [135]:
import random
import string
import secrets
from collections import Counter, defaultdict

from tqdm import tqdm

import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

In [3]:
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
from sklearn.utils import shuffle
import codecs
import re
import collections
import pickle

In [54]:
import torch
from torch.nn import Embedding, Linear, LSTM, Module
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch.optim as optim

In [5]:
from copy import deepcopy
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [6]:
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
from keras import regularizers

In [7]:
col = ['sentiment','tweet_num','date-time','device','user','tweet']

In [8]:
training_1600K = pd.read_csv('./sentiment_data/training.1600000.processed.noemoticon.csv',encoding='ANSI',names=col,header=None)

In [9]:
test_manual_2009_06_14 = pd.read_csv('./sentiment_data/testdata.manual.2009.06.14.csv',names=col,header=None)

In [10]:
test_manual_2009_06_14.head(n=10)

,sentiment,tweet_num,date-time,device,user,tweet
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
5,4,8,Mon May 11 03:22:00 UTC 2009,kindle2,GeorgeVHulme,@richardebaker no. it is too big. I'm quite ha...
6,0,9,Mon May 11 03:22:30 UTC 2009,aig,Seth937,Fuck this economy. I hate aig and their non lo...
7,4,10,Mon May 11 03:26:10 UTC 2009,jquery,dcostalis,Jquery is my new best friend.
8,4,11,Mon May 11 03:27:15 UTC 2009,twitter,PJ_King,Loves twitter
9,4,12,Mon May 11 03:29:20 UTC 2009,obama,mandanicole,how can you not love Obama? he makes jokes abo...


In [11]:
training_1600K.head(n=10)

,sentiment,tweet_num,date-time,device,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [12]:
training_1600K.shape

(1600000, 6)

In [13]:
test_manual_2009_06_14.shape

(498, 6)

In [14]:
training_data = training_1600K[['sentiment','tweet']]

In [15]:
training_data

,sentiment,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [16]:
training_data_shuffled = shuffle(training_data)

In [17]:
def remove_mentions(input_text):
    '''
    Function to remove mentions, preceded by @, in a Pandas Series
    
    Parameters:
        input_text : text to clean
    Output:
        cleaned Pandas Series 
    '''
    return re.sub(r'@\w+', '', input_text)

In [18]:
training_data_rem_mentions = training_data_shuffled[['sentiment','tweet']]

In [19]:
training_data_rem_mentions.tweet = training_data_rem_mentions.tweet.apply(remove_mentions)

In [20]:
training_data_rem_mentions

,sentiment,tweet
1106189,4,goood morning all!
1538699,4,OMG your eyes are so pretty! *-* i &lt;3 iCar...
318404,0,we too. and then i got some moe. and then all...
1258738,4,"We're watching an asian film today, i feel at ..."
428942,0,yeah i feel a bit sickles but im getting bet...
...,...,...
1056564,4,- 14th time? I think you mean 27th time!! So...
382039,0,your a heartbreaker move on so quick
39803,0,Just burn't my croissant under the grill
966577,4,Think it just might be time to Wii.


In [21]:
tokenizer = nltk.RegexpTokenizer(r'\w+')

In [22]:
print(training_data_rem_mentions.iloc[2,1])
print(training_data_rem_mentions.iloc[2,0])

tokenizer.tokenize(training_data_rem_mentions.iloc[2,1] + ' !')

 we too. and then i got some moe. and then all the inappropriate followers were deleted by twitter. so it went from 90-82 
0


['we',
 'too',
 'and',
 'then',
 'i',
 'got',
 'some',
 'moe',
 'and',
 'then',
 'all',
 'the',
 'inappropriate',
 'followers',
 'were',
 'deleted',
 'by',
 'twitter',
 'so',
 'it',
 'went',
 'from',
 '90',
 '82']

In [23]:
MAX_LEN = 140  # Maximum number of words in a sequence
Embed_Dim = 100 # Number of dimensions of the GloVe word embeddings

In [24]:
glove_file_path = './glove.twitter.27B/glove.twitter.27B.' + str(Embed_Dim) + 'd.txt'
emb_dict = {}
glove = codecs.open(glove_file_path ,'r','utf-8')


In [25]:
for line in glove:
    #if (len(line) > 0) :
    try:
        #print(len(line))
        values = line.split()
        #print(len(values))
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        if len(vector) != Embed_Dim :
            print(vector)
            print(word)
            print(len(vector))
        if len(vector) == Embed_Dim :
            emb_dict[word] = vector
    except:
        print('len of line : ' + str(len(line)))
        print('len of values : ' + str(len(line)))
        print('line : ' + str(line))


len of line : 1
len of values : 1
line : 
[-0.73053   -0.15227    0.75504    0.11011    0.14091    0.047278
 -1.0087     0.13282   -0.10939   -0.010336   0.32189   -1.4589
 -0.83385   -0.52429    0.55353    0.054966   0.02489    0.066947
  0.39403   -0.2942    -1.2322    -0.2594    -0.72149    0.3671
  0.24201    0.023268   0.14087    0.60309    0.37282    0.40474
  0.16387    1.5523    -0.28782   -0.26105   -0.83564   -0.031021
  0.26182   -0.093516  -0.36343   -0.10013   -0.113      1.3461
 -0.7571    -0.51527   -0.0099121 -0.34748    0.2534     0.43839
  0.30234    0.0080009  0.51505    0.25082   -0.53778    0.20495
  0.27272    0.13311    0.98437   -0.24143    0.041526   0.21953
 -0.20118    0.068255   1.2481    -0.28648   -0.058264   0.18604
  0.45244    0.36555    0.35107    0.78051   -0.20271    0.99956
 -0.4688    -0.49431   -0.14843    0.0022548 -0.10625   -0.21541
 -0.24243   -0.68123   -0.17896   -0.86271    0.74024    0.73827
  0.4905    -0.71627   -0.49518    0.050481  -0

In [26]:
glove.close()

In [27]:
len(emb_dict)

1193513

In [28]:
list_of_tokens_of_emb_dict = list(emb_dict.keys())

In [29]:
list_of_tokens_of_emb_dict

['<user>',
 '.',
 ':',
 'rt',
 ',',
 '<repeat>',
 '<hashtag>',
 '<number>',
 '<url>',
 '!',
 'i',
 'a',
 '"',
 'the',
 '?',
 'you',
 'to',
 '(',
 '<allcaps>',
 '<elong>',
 ')',
 'me',
 'de',
 '<smile>',
 '！',
 'que',
 'and',
 '。',
 '-',
 'my',
 'no',
 '、',
 'is',
 'it',
 '…',
 'in',
 'n',
 'for',
 '/',
 'of',
 'la',
 "'s",
 '*',
 'do',
 "n't",
 'that',
 'on',
 'y',
 "'",
 'e',
 'o',
 'u',
 'en',
 'this',
 'el',
 'so',
 'be',
 "'m",
 'with',
 'just',
 '>',
 'your',
 '^',
 'like',
 'have',
 'te',
 'at',
 '？',
 'love',
 'se',
 'are',
 '<',
 'm',
 'r',
 'if',
 'all',
 'b',
 '・',
 'not',
 'but',
 'we',
 'es',
 'ya',
 '&',
 'follow',
 'up',
 'what',
 'get',
 'lol',
 'un',
 '♥',
 'lo',
 'when',
 'was',
 '“',
 '”',
 'one',
 'por',
 'si',
 'out',
 '_',
 'mi',
 'can',
 '<sadface>',
 'من',
 '♡',
 '´',
 'he',
 'con',
 'they',
 'now',
 'go',
 '،',
 'para',
 'los',
 'know',
 'haha',
 'good',
 'tu',
 'back',
 '~',
 'about',
 'new',
 ';',
 'as',
 'day',
 'how',
 'who',
 'will',
 'want',
 'people',
 'y

In [30]:
def tweet_to_token_list(tweet):
    token_list = tokenizer.tokenize(tweet)
    lowercased_token_list = [t.lower() for t in token_list]
    embedable_token_list = [t for t in lowercased_token_list if t in list_of_tokens_of_emb_dict]
    return embedable_token_list #np.array(embedable_token_list)

In [31]:
print(tweet_to_token_list(training_data_rem_mentions.iloc[2000,1]))
print(float(int(training_data_rem_mentions.iloc[2000,0])))

['finally', 'this', 'stressful', 'day', 'from', 'hell', 'is', 'over', 'i', 'get', 'to', 'go', 'home', 'only', 'to', 'have', 'to', 'vpn', 'in', 'later', 'to', 'check', 'on', 'some', 'work']
0.0


In [32]:
training_data_rem_mentions

,sentiment,tweet
1106189,4,goood morning all!
1538699,4,OMG your eyes are so pretty! *-* i &lt;3 iCar...
318404,0,we too. and then i got some moe. and then all...
1258738,4,"We're watching an asian film today, i feel at ..."
428942,0,yeah i feel a bit sickles but im getting bet...
...,...,...
1056564,4,- 14th time? I think you mean 27th time!! So...
382039,0,your a heartbreaker move on so quick
39803,0,Just burn't my croissant under the grill
966577,4,Think it just might be time to Wii.


In [33]:
training_data_tokenized = training_data_rem_mentions[['sentiment','tweet']]

In [34]:
training_data_tokenized.sentiment = training_data_tokenized.sentiment.apply(int)
training_data_tokenized.sentiment = training_data_tokenized.sentiment.apply(float)
training_data_tokenized.sentiment = training_data_tokenized.sentiment/4.0

In [35]:
training_data_tokenized.head(n=25)

,sentiment,tweet
1106189,1.0,goood morning all!
1538699,1.0,OMG your eyes are so pretty! *-* i &lt;3 iCar...
318404,0.0,we too. and then i got some moe. and then all...
1258738,1.0,"We're watching an asian film today, i feel at ..."
428942,0.0,yeah i feel a bit sickles but im getting bet...
1451773,1.0,is watching TFC haha
1281115,1.0,"Nah, that's nothing to do with your skill as ..."
1033705,1.0,Good night all. Sweet dreams and I'll talk to...
506720,0.0,ahhhh *switches it on* did i miss Conan? i de...
1315958,1.0,"when I show it, use PC not Mac"


In [36]:
len(tweet_to_token_list(training_data_tokenized.tweet.iloc[71]))

2

In [37]:
training_data_tokenized.iloc[71]

sentiment             1.0
tweet         It works!  
Name: 1250540, dtype: object

In [38]:
def tweet_to_word_vectors(tweet):
  processed_list_of_tokens = tweet_to_token_list(tweet)

  vectors = []

  for token in processed_list_of_tokens:
    token_vector = emb_dict[token]
    vectors.append(token_vector)
  
  return np.array(vectors, dtype=float)

In [39]:
tweet_to_word_vectors(training_data_tokenized.tweet.iloc[71]).shape

(2, 100)

In [40]:
def df_to_X_y(df):
  y = df['sentiment']

  all_word_vector_sequences = []

  for tweet in df['tweet']:
    tweet_as_word_vector = tweet_to_word_vectors(tweet)
    
    if tweet_as_word_vector.shape[0] == 0:
      tweet_as_word_vector = np.zeros(shape=(1, Embed_Dim))

    all_word_vector_sequences.append(tweet_as_word_vector)
  
  return all_word_vector_sequences, y

# Saving the x and y data in chunks

In [ ]:
list_of_x = []
list_of_y = []
for i in range(320):
    start_time = time.time()
    x , y = df_to_X_y(training_data_tokenized[i*5000:5000*(i+1)])
    with open('./Chunks/x'+str(i+1)+ '.pickle', 'wb') as file:
        pickle.dump(x, file)
    with open('./Chunks/y'+str(i+1)+ '.pickle', 'wb') as file:
        pickle.dump(y, file)

    list_of_x.append(x)
    list_of_y.append(y)
    print('time taken in sec = ' + str(time.time() - start_time) + 'for chunk '+ str(i+1))

In [41]:
len(list_of_x)

320

In [42]:
len(list_of_y)

320

In [49]:
len(list_of_x)

320

# Loading the x and y data from chunks

In [41]:
#list_of_x = []
list_of_y = []
for i in range(320):
    start_time = time.time()
    #x = []
    y = []
    #with open('./Chunks/x'+str(i+1)+ '.pickle', 'rb') as file:
    #    x = pickle.load(file)
    with open('./Chunks/y'+str(i+1)+ '.pickle', 'rb') as file:
        y = pickle.load(file)

    #list_of_x.append(x)
    list_of_y.append(y)
    print('time taken in sec = ' + str(time.time() - start_time) + 'for chunk '+ str(i+1))

time taken in sec = 0.0009996891021728516for chunk 1
time taken in sec = 0.0009975433349609375for chunk 2
time taken in sec = 0.0009958744049072266for chunk 3
time taken in sec = 0.0021562576293945312for chunk 4
time taken in sec = 0.0012645721435546875for chunk 5
time taken in sec = 0.0010061264038085938for chunk 6
time taken in sec = 0.0010020732879638672for chunk 7
time taken in sec = 0.0021209716796875for chunk 8
time taken in sec = 0.0010068416595458984for chunk 9
time taken in sec = 0.0011086463928222656for chunk 10
time taken in sec = 0.0012362003326416016for chunk 11
time taken in sec = 0.0010564327239990234for chunk 12
time taken in sec = 0.001214742660522461for chunk 13
time taken in sec = 0.0010044574737548828for chunk 14
time taken in sec = 0.0019981861114501953for chunk 15
time taken in sec = 0.0013813972473144531for chunk 16
time taken in sec = 0.0010051727294921875for chunk 17
time taken in sec = 0.0for chunk 18
time taken in sec = 0.0012426376342773438for chunk 19
time 

time taken in sec = 0.0019927024841308594for chunk 168
time taken in sec = 0.000997304916381836for chunk 169
time taken in sec = 0.0009984970092773438for chunk 170
time taken in sec = 0.000997304916381836for chunk 171
time taken in sec = 0.0009944438934326172for chunk 172
time taken in sec = 0.001995086669921875for chunk 173
time taken in sec = 0.0009944438934326172for chunk 174
time taken in sec = 0.0009970664978027344for chunk 175
time taken in sec = 0.0009963512420654297for chunk 176
time taken in sec = 0.0009965896606445312for chunk 177
time taken in sec = 0.0010178089141845703for chunk 178
time taken in sec = 0.0009756088256835938for chunk 179
time taken in sec = 0.0010042190551757812for chunk 180
time taken in sec = 0.000989675521850586for chunk 181
time taken in sec = 0.0019948482513427734for chunk 182
time taken in sec = 0.0009953975677490234for chunk 183
time taken in sec = 0.0009963512420654297for chunk 184
time taken in sec = 0.0009980201721191406for chunk 185
time taken in 

# Data splitting

In [42]:
def pad2_X(X, desired_sequence_length=140):
    X_copy = deepcopy(X)
    start_time = time.time()
    for i, x in enumerate(X):
        if (x.shape[1] != Embed_Dim):
            print('Shape of exception causing x = ' + str(x.shape))
            x_new = np.zeros(shape=(1, Embed_Dim))
            x_seq_len = x_new.shape[0]
            sequence_length_difference = desired_sequence_length - x_seq_len
            pad = np.zeros(shape=(sequence_length_difference, Embed_Dim))
            X_copy[i] = np.concatenate([x_new, pad])
            if ((i%100) == 0):
                print('time taken in sec = ' + str(time.time() - start_time) + 'for '+ str(i+1))
        else :
            x_seq_len = x.shape[0]
            sequence_length_difference = desired_sequence_length - x_seq_len
            pad = np.zeros(shape=(sequence_length_difference, Embed_Dim))
            X_copy[i] = np.concatenate([x, pad])
            if ((i%100) == 0):
                print('time taken in sec = ' + str(time.time() - start_time) + 'for '+ str(i+1))
                
    return np.array(X_copy).astype(float)

In [43]:
len(list_of_x)

NameError: name 'list_of_x' is not defined

In [44]:
len(list_of_y)

320

In [45]:
list_of_y_nparray =[]
for y in list_of_y:
    y_nparray = np.asarray(y,dtype = int)
    list_of_y_nparray.append(y_nparray)

In [46]:
list_of_y_nparray

[array([0, 1, 0, ..., 0, 0, 0]),
 array([0, 1, 0, ..., 1, 0, 1]),
 array([0, 1, 1, ..., 1, 1, 0]),
 array([1, 1, 1, ..., 0, 1, 1]),
 array([1, 0, 0, ..., 1, 1, 0]),
 array([0, 1, 0, ..., 0, 1, 1]),
 array([1, 1, 1, ..., 1, 0, 1]),
 array([0, 1, 0, ..., 0, 1, 1]),
 array([0, 0, 1, ..., 0, 1, 0]),
 array([1, 0, 1, ..., 0, 0, 1]),
 array([0, 0, 1, ..., 1, 0, 0]),
 array([1, 0, 1, ..., 0, 1, 1]),
 array([0, 1, 0, ..., 1, 1, 1]),
 array([1, 0, 0, ..., 1, 0, 0]),
 array([0, 1, 1, ..., 1, 1, 0]),
 array([0, 1, 0, ..., 0, 1, 1]),
 array([0, 0, 0, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 0]),
 array([0, 0, 1, ..., 0, 1, 1]),
 array([1, 1, 1, ..., 0, 1, 0]),
 array([0, 1, 1, ..., 0, 1, 1]),
 array([1, 0, 1, ..., 1, 0, 1]),
 array([1, 1, 0, ..., 1, 0, 0]),
 array([0, 1, 1, ..., 0, 1, 0]),
 array([0, 1, 1, ..., 1, 1, 0]),
 array([1, 0, 1, ..., 0, 1, 1]),
 array([0, 1, 0, ..., 0, 0, 1]),
 array([1, 1, 1, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 1, 0, 0]),
 array([0, 1, 1, ..., 0, 1, 1]),
 array([0,

In [61]:
#list_of_x = []
for i in range(320):
    start_time = time.time()
    x = []
    with open('./Chunks/x'+str(i+1)+ '.pickle', 'rb') as file:
        x = pickle.load(file)
    x_paded = pad2_X(x,desired_sequence_length=MAX_LEN)
    with open('./Chucks_paded/x_paded'+str(i+1)+ '.pickle', 'wb') as file:
        pickle.dump(x_paded, file)
    print('time taken in sec = ' + str(time.time() - start_time) + 'for chunk '+ str(i+1))

time taken in sec = 0.0for 1
time taken in sec = 0.00600123405456543for 101
time taken in sec = 0.011980056762695312for 201
time taken in sec = 0.018959522247314453for 301
time taken in sec = 0.023920059204101562for 401
time taken in sec = 0.02892470359802246for 501
time taken in sec = 0.033890724182128906for 601
time taken in sec = 0.03889656066894531for 701
time taken in sec = 0.04388880729675293for 801
time taken in sec = 0.04884171485900879for 901
time taken in sec = 0.05388522148132324for 1001
time taken in sec = 0.05880308151245117for 1101
time taken in sec = 0.06378626823425293for 1201
time taken in sec = 0.06976628303527832for 1301
time taken in sec = 0.07476997375488281for 1401
time taken in sec = 0.08078122138977051for 1501
time taken in sec = 0.08578681945800781for 1601
time taken in sec = 0.0912322998046875for 1701
time taken in sec = 0.09725189208984375for 1801
time taken in sec = 0.10319232940673828for 1901
time taken in sec = 0.10825395584106445for 2001
time taken in sec

time taken in sec = 0.10924482345581055for 2201
time taken in sec = 0.10924482345581055for 2301
time taken in sec = 0.12486910820007324for 2401
time taken in sec = 0.12486910820007324for 2501
time taken in sec = 0.12486910820007324for 2601
time taken in sec = 0.14049196243286133for 2701
time taken in sec = 0.14699602127075195for 2801
time taken in sec = 0.15650677680969238for 2901
time taken in sec = 0.15650677680969238for 3001
time taken in sec = 0.15650677680969238for 3101
time taken in sec = 0.1721651554107666for 3201
time taken in sec = 0.1721651554107666for 3301
time taken in sec = 0.1721651554107666for 3401
time taken in sec = 0.18777108192443848for 3501
time taken in sec = 0.18777108192443848for 3601
time taken in sec = 0.18777108192443848for 3701
time taken in sec = 0.20339322090148926for 3801
time taken in sec = 0.20339322090148926for 3901
time taken in sec = 0.20339322090148926for 4001
time taken in sec = 0.2190396785736084for 4101
time taken in sec = 0.2190396785736084for 42

time taken in sec = 1.2810032367706299for chunk 7
time taken in sec = 0.0for 1
time taken in sec = 0.002990245819091797for 101
time taken in sec = 0.004983425140380859for 201
time taken in sec = 0.0069768428802490234for 301
time taken in sec = 0.008970499038696289for 401
time taken in sec = 0.010967254638671875for 501
time taken in sec = 0.012964010238647461for 601
time taken in sec = 0.014959096908569336for 701
time taken in sec = 0.016950368881225586for 801
time taken in sec = 0.01894402503967285for 901
time taken in sec = 0.020937442779541016for 1001
time taken in sec = 0.02293229103088379for 1101
time taken in sec = 0.024925708770751953for 1201
time taken in sec = 0.02831125259399414for 1301
time taken in sec = 0.03130006790161133for 1401
time taken in sec = 0.03336501121520996for 1501
time taken in sec = 0.036284685134887695for 1601
time taken in sec = 0.03928637504577637for 1701
time taken in sec = 0.042263031005859375for 1801
time taken in sec = 0.04525279998779297for 1901
time 

time taken in sec = 0.10983824729919434for 3601
time taken in sec = 0.10983824729919434for 3701
time taken in sec = 0.1255016326904297for 3801
time taken in sec = 0.1255016326904297for 3901
time taken in sec = 0.1255016326904297for 4001
time taken in sec = 0.14107394218444824for 4101
time taken in sec = 0.14107394218444824for 4201
time taken in sec = 0.15259885787963867for 4301
time taken in sec = 0.15718460083007812for 4401
time taken in sec = 0.15718460083007812for 4501
time taken in sec = 0.15718460083007812for 4601
time taken in sec = 0.172837495803833for 4701
time taken in sec = 0.172837495803833for 4801
time taken in sec = 0.172837495803833for 4901
time taken in sec = 1.246903419494629for chunk 11
time taken in sec = 0.0for 1
time taken in sec = 0.0for 101
time taken in sec = 0.01563549041748047for 201
time taken in sec = 0.01563549041748047for 301
time taken in sec = 0.01563549041748047for 401
time taken in sec = 0.031287193298339844for 501
time taken in sec = 0.0312871932983398

time taken in sec = 0.14139246940612793for 4101
time taken in sec = 0.14139246940612793for 4201
time taken in sec = 0.15702009201049805for 4301
time taken in sec = 0.15702009201049805for 4401
time taken in sec = 0.15702009201049805for 4501
time taken in sec = 0.17264294624328613for 4601
time taken in sec = 0.17264294624328613for 4701
time taken in sec = 0.18826723098754883for 4801
time taken in sec = 0.18826723098754883for 4901
time taken in sec = 1.154939889907837for chunk 15
time taken in sec = 0.0for 1
time taken in sec = 0.0for 101
time taken in sec = 0.0for 201
time taken in sec = 0.0for 301
time taken in sec = 0.0for 401
time taken in sec = 0.0for 501
time taken in sec = 0.015560150146484375for 601
time taken in sec = 0.015560150146484375for 701
time taken in sec = 0.015560150146484375for 801
time taken in sec = 0.022075653076171875for 901
time taken in sec = 0.022075653076171875for 1001
time taken in sec = 0.022075653076171875for 1101
time taken in sec = 0.022075653076171875for 

...

time taken in sec = 0.08917021751403809for 2501
time taken in sec = 0.09414339065551758for 2601
time taken in sec = 0.09917712211608887for 2701
time taken in sec = 0.1041555404663086for 2801
time taken in sec = 0.10910725593566895for 2901
time taken in sec = 0.11407589912414551for 3001
time taken in sec = 0.11905908584594727for 3101
time taken in sec = 0.12503933906555176for 3201
time taken in sec = 0.13004422187805176for 3301
time taken in sec = 0.1350102424621582for 3401
time taken in sec = 0.14297914505004883for 3501
time taken in sec = 0.14995574951171875for 3601
time taken in sec = 0.15693259239196777for 3701
time taken in sec = 0.1640019416809082for 3801
time taken in sec = 0.17191028594970703for 3901
time taken in sec = 0.17927098274230957for 4001
time taken in sec = 0.18724560737609863for 4101
time taken in sec = 0.20020484924316406for 4201
time taken in sec = 0.21118950843811035for 4301
time taken in sec = 0.21913671493530273for 4401
time taken in sec = 0.22611427307128906for 

time taken in sec = 0.34208083152770996for 4801
time taken in sec = 0.34957313537597656for 4901
time taken in sec = 1.5864732265472412for chunk 309
time taken in sec = 0.0for 1
time taken in sec = 0.0039865970611572266for 101
time taken in sec = 0.004983425140380859for 201
time taken in sec = 0.007973432540893555for 301
time taken in sec = 0.009966611862182617for 401
time taken in sec = 0.012956857681274414for 501
time taken in sec = 0.01594710350036621for 601
time taken in sec = 0.017940282821655273for 701
time taken in sec = 0.02093052864074707for 801
time taken in sec = 0.023922443389892578for 901
time taken in sec = 0.026912450790405273for 1001
time taken in sec = 0.02990245819091797for 1101
time taken in sec = 0.03289318084716797for 1201
time taken in sec = 0.035883188247680664for 1301
time taken in sec = 0.039374589920043945for 1401
time taken in sec = 0.043363332748413086for 1501
time taken in sec = 0.04635334014892578for 1601
time taken in sec = 0.050339698791503906for 1701
tim

time taken in sec = 0.13361096382141113for 3401
time taken in sec = 0.14160966873168945for 3501
time taken in sec = 0.14960432052612305for 3601
time taken in sec = 0.15755701065063477for 3701
time taken in sec = 0.1645348072052002for 3801
time taken in sec = 0.1715090274810791for 3901
time taken in sec = 0.17848753929138184for 4001
time taken in sec = 0.18546319007873535for 4101
time taken in sec = 0.19443440437316895for 4201
time taken in sec = 0.2024068832397461for 4301
time taken in sec = 0.20938658714294434for 4401
time taken in sec = 0.21636319160461426for 4501
time taken in sec = 0.2228529453277588for 4601
time taken in sec = 0.22883296012878418for 4701
time taken in sec = 0.2358095645904541for 4801
time taken in sec = 0.24278640747070312for 4901
time taken in sec = 1.4195027351379395for chunk 313
time taken in sec = 0.0for 1
time taken in sec = 0.006551265716552734for 101
time taken in sec = 0.012530326843261719for 201
time taken in sec = 0.014524698257446289for 301
time taken i

time taken in sec = 0.14170551300048828for 2201
time taken in sec = 0.15084481239318848for 2301
time taken in sec = 0.15881848335266113for 2401
time taken in sec = 0.16679143905639648for 2501
time taken in sec = 0.17476487159729004for 2601
time taken in sec = 0.18373513221740723for 2701
time taken in sec = 0.19170808792114258for 2801
time taken in sec = 0.19992518424987793for 2901
time taken in sec = 0.20909857749938965for 3001
time taken in sec = 0.21828222274780273for 3101
time taken in sec = 0.22649550437927246for 3201
time taken in sec = 0.2337346076965332for 3301
time taken in sec = 0.24186182022094727for 3401
time taken in sec = 0.24796319007873535for 3501
time taken in sec = 0.2532801628112793for 3601
time taken in sec = 0.2597174644470215for 3701
time taken in sec = 0.26572656631469727for 3801
time taken in sec = 0.27133822441101074for 3901
time taken in sec = 0.2779202461242676for 4001
time taken in sec = 0.2836909294128418for 4101
time taken in sec = 0.29070234298706055for 42

time taken in sec = 1.5401496887207031for chunk 320


In [47]:
list_of_y_nparray[0]

array([0, 1, 0, ..., 0, 0, 0])

In [49]:
 x_example = []
with open('./Chucks_paded/x_paded'+str(1)+ '.pickle', 'rb') as file:
    x_example = pickle.load(file)

In [50]:
len(x_example)

5000

In [51]:
x_example_nparr  = np.asarray(x_example)

In [52]:
x_example_nparr.shape

(5000, 140, 100)

# Model

## Pytorch Model

In [100]:
class PyTorchModel(nn.Module):
    def __init__(self, max_len, embed_dim):
        super(PyTorchModel, self).__init__()
        self.model = nn.Sequential(
            nn.LSTM(input_size=embed_dim, hidden_size=64, batch_first=True, bidirectional=False, num_layers=1),
            nn.Dropout(0.2),
            nn.LSTM(input_size=64, hidden_size=64, batch_first=True, bidirectional=False, num_layers=1),
            nn.Dropout(0.2),
            nn.LSTM(input_size=64, hidden_size=32, batch_first=True, bidirectional=False, num_layers=1),
            nn.Dropout(0.2),
            nn.Flatten(),
            nn.Linear(32 * max_len, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x, _ = self.model[0](x)
        x = self.model[1](x)
        x, _ = self.model[2](x)
        x = self.model[3](x)
        x, _ = self.model[4](x)
        x = self.model[5](x)
        x = self.model[6](x)
        x = self.model[7](x)
        x = self.model[8](x)
        return x

# Instantiate the PyTorch model
pytorch_model = PyTorchModel(MAX_LEN, Embed_Dim)

# Move the model to a specific device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print()
pytorch_model.to(device)

# Optionally, you can print the model to see its architecture
print(pytorch_model)

cpu

PyTorchModel(
  (model): Sequential(
    (0): LSTM(100, 64, batch_first=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): LSTM(64, 64, batch_first=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): LSTM(64, 32, batch_first=True)
    (5): Dropout(p=0.2, inplace=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=4480, out_features=1, bias=True)
    (8): Sigmoid()
  )
)


In [101]:
criterion = nn.BCELoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.01)
num_epochs = 20
batch_size = 32
num_batches = len(X_train) // batch_size

In [102]:
# Learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
# Early stopping parameters
early_stopping_patience = 5
best_val_loss = float('inf')
patience_counter = 0

In [108]:
def Train(X_train,X_val,y_train,y_val,best_val_loss,patience_counter):
    start_time = time.time()
    for epoch in range(num_epochs):
        running_loss = 0.0
        pytorch_model.train()  # Set the model to train mode

        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size

            # Get the current batch of data and labels
            inputs = X_train[start_idx:end_idx].to(device)
            labels = y_train[start_idx:end_idx].to(device)

            # Convert labels to the correct shape
            labels = labels.unsqueeze(1)  # Add an extra dimension
        
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = pytorch_model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()

            # Update model parameters
            optimizer.step()

            # Print statistics
            running_loss += loss.item()
            if batch_idx % 100 == 99:    # Print every 100 mini-batches
                print(f"[Epoch {epoch + 1}, Batch {batch_idx + 1}] Loss: {running_loss / 100:.3f}")
                running_loss = 0.0

        # Validation
        pytorch_model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            val_loss = 0.0
            num_val_batches = len(X_val) // batch_size
            for batch_idx in range(num_val_batches):
                start_idx = batch_idx * batch_size
                end_idx = start_idx + batch_size

                inputs = X_val[start_idx:end_idx].to(device)
                labels = y_val[start_idx:end_idx].to(device)

                # Convert labels to the correct shape
                labels = labels.unsqueeze(1)  # Add an extra dimension

                outputs = pytorch_model(inputs)
                val_loss += criterion(outputs, labels).item()

            val_loss /= num_val_batches

            # Learning rate scheduling
            lr_scheduler.step()

            # Early stopping check
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                print('best_val_loss = ' + str(best_val_loss))
                # Save the model if needed
                torch.save(pytorch_model.state_dict(), 'best_model.pt')
                print("model saved as best_model.pt.")
            else:
                patience_counter += 1
                if patience_counter >= early_stopping_patience:
                    print("Early stopping triggered.")
                    break

    
    print("Training finished.")
    print('time taken in sec = ' + str(time.time() - start_time) + ' for Training. \n\n\n\n')

In [109]:
for i in range(320):
    x_example = []
    with open('./Chucks_paded/x_paded'+str(i+1)+ '.pickle', 'rb') as file:
        x_example = pickle.load(file)
    x_example_nparr  = np.asarray(x_example)
    X_train = x_example_nparr[:4000]
    X_val = x_example_nparr[4000:]
    y_train = list_of_y_nparray[i][:4000]
    y_val = list_of_y_nparray[i][4000:]
    X_train = torch.tensor(X_train,dtype=torch.float32)
    X_val = torch.tensor(X_val,dtype=torch.float32)
    y_train = torch.tensor(y_train,dtype=torch.float32)
    y_val = torch.tensor(y_val,dtype=torch.float32)
    print('Training on the chunk : '+'./Chucks_paded/x_paded'+str(i+1)+ '.pickle')
    Train(X_train,X_val,y_train,y_val,best_val_loss,patience_counter)

Training on the chunk : ./Chucks_paded/x_paded1.pickle
[Epoch 1, Batch 100] Loss: 0.266
best_val_loss = 0.7031085241225458
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.266
[Epoch 3, Batch 100] Loss: 0.269
[Epoch 4, Batch 100] Loss: 0.266
[Epoch 5, Batch 100] Loss: 0.266
[Epoch 6, Batch 100] Loss: 0.267
Early stopping triggered.
Training finished.
time taken in sec = 35.70338439941406 for Training. 




Training on the chunk : ./Chucks_paded/x_paded2.pickle
[Epoch 1, Batch 100] Loss: 0.617
best_val_loss = 0.60800483534413
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.622
best_val_loss = 0.607982688373135
model saved as best_model.pt.
[Epoch 3, Batch 100] Loss: 0.620
best_val_loss = 0.6079635456685097
model saved as best_model.pt.
[Epoch 4, Batch 100] Loss: 0.622
best_val_loss = 0.6079633476272706
model saved as best_model.pt.
[Epoch 5, Batch 100] Loss: 0.626
best_val_loss = 0.6079630476813163
model saved as best_model.pt.
[Epoch 6, Batch 100] Loss: 0.621
best_

Training on the chunk : ./Chucks_paded/x_paded19.pickle
[Epoch 1, Batch 100] Loss: 0.652
best_val_loss = 0.6853445777969975
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.653
[Epoch 3, Batch 100] Loss: 0.654
[Epoch 4, Batch 100] Loss: 0.657
[Epoch 5, Batch 100] Loss: 0.654
[Epoch 6, Batch 100] Loss: 0.656
Early stopping triggered.
Training finished.
time taken in sec = 36.382262229919434 for Training. 




Training on the chunk : ./Chucks_paded/x_paded20.pickle
[Epoch 1, Batch 100] Loss: 0.635
best_val_loss = 0.6364369623122677
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.633
[Epoch 3, Batch 100] Loss: 0.633
[Epoch 4, Batch 100] Loss: 0.635
[Epoch 5, Batch 100] Loss: 0.635
[Epoch 6, Batch 100] Loss: 0.632
Early stopping triggered.
Training finished.
time taken in sec = 34.68354797363281 for Training. 




Training on the chunk : ./Chucks_paded/x_paded21.pickle
[Epoch 1, Batch 100] Loss: 0.652
best_val_loss = 0.6584485168418577
model saved as best_model.pt.
[Ep

[Epoch 4, Batch 100] Loss: 0.625
[Epoch 5, Batch 100] Loss: 0.618
[Epoch 6, Batch 100] Loss: 0.622
Early stopping triggered.
Training finished.
time taken in sec = 35.89904999732971 for Training. 




Training on the chunk : ./Chucks_paded/x_paded39.pickle
[Epoch 1, Batch 100] Loss: 0.599
best_val_loss = 0.6256843324630491
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.605
[Epoch 3, Batch 100] Loss: 0.602
[Epoch 4, Batch 100] Loss: 0.608
[Epoch 5, Batch 100] Loss: 0.601
[Epoch 6, Batch 100] Loss: 0.604
Early stopping triggered.
Training finished.
time taken in sec = 35.575796365737915 for Training. 




Training on the chunk : ./Chucks_paded/x_paded40.pickle
[Epoch 1, Batch 100] Loss: 0.617
best_val_loss = 0.6102038112378889
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.618
[Epoch 3, Batch 100] Loss: 0.609
[Epoch 4, Batch 100] Loss: 0.615
[Epoch 5, Batch 100] Loss: 0.611
[Epoch 6, Batch 100] Loss: 0.618
Early stopping triggered.
Training finished.
time taken in

Training on the chunk : ./Chucks_paded/x_paded58.pickle
[Epoch 1, Batch 100] Loss: 0.651
best_val_loss = 0.6635878172612959
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.648
[Epoch 3, Batch 100] Loss: 0.644
[Epoch 4, Batch 100] Loss: 0.643
[Epoch 5, Batch 100] Loss: 0.649
[Epoch 6, Batch 100] Loss: 0.646
Early stopping triggered.
Training finished.
time taken in sec = 35.41550588607788 for Training. 




Training on the chunk : ./Chucks_paded/x_paded59.pickle
[Epoch 1, Batch 100] Loss: 0.601
best_val_loss = 0.60261482192624
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.601
[Epoch 3, Batch 100] Loss: 0.605
[Epoch 4, Batch 100] Loss: 0.605
[Epoch 5, Batch 100] Loss: 0.606
[Epoch 6, Batch 100] Loss: 0.606
Early stopping triggered.
Training finished.
time taken in sec = 35.27658939361572 for Training. 




Training on the chunk : ./Chucks_paded/x_paded60.pickle
[Epoch 1, Batch 100] Loss: 0.578
best_val_loss = 0.6544540976324389
model saved as best_model.pt.
[Epoch

[Epoch 4, Batch 100] Loss: 0.679
[Epoch 5, Batch 100] Loss: 0.673
[Epoch 6, Batch 100] Loss: 0.677
Early stopping triggered.
Training finished.
time taken in sec = 35.289302110672 for Training. 




Training on the chunk : ./Chucks_paded/x_paded78.pickle
[Epoch 1, Batch 100] Loss: 0.660
best_val_loss = 0.6431416946072732
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.662
[Epoch 3, Batch 100] Loss: 0.664
[Epoch 4, Batch 100] Loss: 0.661
[Epoch 5, Batch 100] Loss: 0.666
[Epoch 6, Batch 100] Loss: 0.664
Early stopping triggered.
Training finished.
time taken in sec = 35.11251616477966 for Training. 




Training on the chunk : ./Chucks_paded/x_paded79.pickle
[Epoch 1, Batch 100] Loss: 0.654
best_val_loss = 0.5988589900155221
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.632
[Epoch 3, Batch 100] Loss: 0.630
[Epoch 4, Batch 100] Loss: 0.633
[Epoch 5, Batch 100] Loss: 0.634
[Epoch 6, Batch 100] Loss: 0.629
Early stopping triggered.
Training finished.
time taken in se

...

[Epoch 3, Batch 100] Loss: 0.611
[Epoch 4, Batch 100] Loss: 0.609
[Epoch 5, Batch 100] Loss: 0.612
[Epoch 6, Batch 100] Loss: 0.615
Early stopping triggered.
Training finished.
time taken in sec = 35.20868706703186 for Training. 




Training on the chunk : ./Chucks_paded/x_paded234.pickle
[Epoch 1, Batch 100] Loss: 0.652
best_val_loss = 0.5558296009417503
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.651
[Epoch 3, Batch 100] Loss: 0.649
[Epoch 4, Batch 100] Loss: 0.643
[Epoch 5, Batch 100] Loss: 0.651
[Epoch 6, Batch 100] Loss: 0.648
Early stopping triggered.
Training finished.
time taken in sec = 35.23818492889404 for Training. 




Training on the chunk : ./Chucks_paded/x_paded235.pickle
[Epoch 1, Batch 100] Loss: 0.593
best_val_loss = 0.665202461904095
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.594
[Epoch 3, Batch 100] Loss: 0.599
[Epoch 4, Batch 100] Loss: 0.592
[Epoch 5, Batch 100] Loss: 0.597
[Epoch 6, Batch 100] Loss: 0.592
Early stopping triggered.

Training on the chunk : ./Chucks_paded/x_paded253.pickle
[Epoch 1, Batch 100] Loss: 0.632
best_val_loss = 0.6579755184150511
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.635
[Epoch 3, Batch 100] Loss: 0.632
[Epoch 4, Batch 100] Loss: 0.635
[Epoch 5, Batch 100] Loss: 0.641
[Epoch 6, Batch 100] Loss: 0.630
Early stopping triggered.
Training finished.
time taken in sec = 35.765663146972656 for Training. 




Training on the chunk : ./Chucks_paded/x_paded254.pickle
[Epoch 1, Batch 100] Loss: 0.638
best_val_loss = 0.6317122376734211
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.643
[Epoch 3, Batch 100] Loss: 0.635
[Epoch 4, Batch 100] Loss: 0.643
[Epoch 5, Batch 100] Loss: 0.644
[Epoch 6, Batch 100] Loss: 0.643
Early stopping triggered.
Training finished.
time taken in sec = 35.922757387161255 for Training. 




Training on the chunk : ./Chucks_paded/x_paded255.pickle
[Epoch 1, Batch 100] Loss: 0.592
best_val_loss = 0.6512338759437684
model saved as best_model.pt.

[Epoch 3, Batch 100] Loss: 0.644
[Epoch 4, Batch 100] Loss: 0.640
[Epoch 5, Batch 100] Loss: 0.644
[Epoch 6, Batch 100] Loss: 0.644
Early stopping triggered.
Training finished.
time taken in sec = 35.10947060585022 for Training. 




Training on the chunk : ./Chucks_paded/x_paded273.pickle
[Epoch 1, Batch 100] Loss: 0.605
best_val_loss = 0.7376965563143453
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.605
[Epoch 3, Batch 100] Loss: 0.601
[Epoch 4, Batch 100] Loss: 0.605
[Epoch 5, Batch 100] Loss: 0.604
[Epoch 6, Batch 100] Loss: 0.610
Early stopping triggered.
Training finished.
time taken in sec = 35.3729043006897 for Training. 




Training on the chunk : ./Chucks_paded/x_paded274.pickle
[Epoch 1, Batch 100] Loss: 0.635
best_val_loss = 0.7644841199921023
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.647
[Epoch 3, Batch 100] Loss: 0.647
[Epoch 4, Batch 100] Loss: 0.645
[Epoch 5, Batch 100] Loss: 0.644
[Epoch 6, Batch 100] Loss: 0.646
Early stopping triggered.

Training on the chunk : ./Chucks_paded/x_paded292.pickle
[Epoch 1, Batch 100] Loss: 0.632
best_val_loss = 0.5989324796584344
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.642
[Epoch 3, Batch 100] Loss: 0.634
[Epoch 4, Batch 100] Loss: 0.639
[Epoch 5, Batch 100] Loss: 0.635
[Epoch 6, Batch 100] Loss: 0.640
Early stopping triggered.
Training finished.
time taken in sec = 35.32698345184326 for Training. 




Training on the chunk : ./Chucks_paded/x_paded293.pickle
[Epoch 1, Batch 100] Loss: 0.630
best_val_loss = 0.6057114754953692
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.639
[Epoch 3, Batch 100] Loss: 0.628
[Epoch 4, Batch 100] Loss: 0.630
[Epoch 5, Batch 100] Loss: 0.636
[Epoch 6, Batch 100] Loss: 0.636
Early stopping triggered.
Training finished.
time taken in sec = 35.59112215042114 for Training. 




Training on the chunk : ./Chucks_paded/x_paded294.pickle
[Epoch 1, Batch 100] Loss: 0.646
best_val_loss = 0.5947910763563649
model saved as best_model.pt.
[

[Epoch 3, Batch 100] Loss: 0.599
[Epoch 4, Batch 100] Loss: 0.592
[Epoch 5, Batch 100] Loss: 0.598
[Epoch 6, Batch 100] Loss: 0.597
Early stopping triggered.
Training finished.
time taken in sec = 35.40180969238281 for Training. 




Training on the chunk : ./Chucks_paded/x_paded312.pickle
[Epoch 1, Batch 100] Loss: 0.603
best_val_loss = 0.5611583268450152
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.608
[Epoch 3, Batch 100] Loss: 0.600
[Epoch 4, Batch 100] Loss: 0.596
[Epoch 5, Batch 100] Loss: 0.599
[Epoch 6, Batch 100] Loss: 0.603
Early stopping triggered.
Training finished.
time taken in sec = 35.34679102897644 for Training. 




Training on the chunk : ./Chucks_paded/x_paded313.pickle
[Epoch 1, Batch 100] Loss: 0.606
best_val_loss = 0.5781366546307841
model saved as best_model.pt.
[Epoch 2, Batch 100] Loss: 0.606
[Epoch 3, Batch 100] Loss: 0.599
[Epoch 4, Batch 100] Loss: 0.601
[Epoch 5, Batch 100] Loss: 0.610
[Epoch 6, Batch 100] Loss: 0.603
Early stopping triggered

In [132]:
def Evaluate(string):
    cleaned_up_string = remove_mentions(string)
    list_of_tweet =[]
    list_of_tweet.append(tweet_to_word_vectors(cleaned_up_string))
    x_paded = pad2_X(list_of_tweet,desired_sequence_length=MAX_LEN)
    x_paded_tensor = torch.tensor(x_paded, dtype=torch.float32).to(device)  # Convert and move to device
    print(x_paded_tensor.shape)
    pytorch_model.eval()
    y = pytorch_model(x_paded_tensor)
    print(y.shape)
    print(y)
    
    return 4*y

In [146]:
Evaluate('hey evryone')

time taken in sec = 0.0for 1
torch.Size([1, 140, 100])
torch.Size([1, 1])
tensor([[0.9412]], grad_fn=<SigmoidBackward0>)


tensor([[3.7647]], grad_fn=<MulBackward0>)

In [147]:
Evaluate('everything sucks')

time taken in sec = 0.0009772777557373047for 1
torch.Size([1, 140, 100])
torch.Size([1, 1])
tensor([[0.0137]], grad_fn=<SigmoidBackward0>)


tensor([[0.0548]], grad_fn=<MulBackward0>)

In [150]:
Evaluate('last night\'s electoral debate was the historic')

time taken in sec = 0.0for 1
torch.Size([1, 140, 100])
torch.Size([1, 1])
tensor([[0.9836]], grad_fn=<SigmoidBackward0>)


tensor([[3.9345]], grad_fn=<MulBackward0>)

In [149]:
Evaluate('last night\'s electoral debate was the historically bad')

time taken in sec = 0.0for 1
torch.Size([1, 140, 100])
torch.Size([1, 1])
tensor([[0.2937]], grad_fn=<SigmoidBackward0>)


tensor([[1.1747]], grad_fn=<MulBackward0>)

In [145]:
Evaluate('सब कुछ ख़राब हो रहा है')

time taken in sec = 0.0for 1
torch.Size([1, 140, 100])
torch.Size([1, 1])
tensor([[0.0522]], grad_fn=<SigmoidBackward0>)


tensor([[0.2087]], grad_fn=<MulBackward0>)

In [144]:
Evaluate('सब कुछ अच्छा चल रहा है')

time taken in sec = 0.0for 1
torch.Size([1, 140, 100])
torch.Size([1, 1])
tensor([[0.0193]], grad_fn=<SigmoidBackward0>)


tensor([[0.0771]], grad_fn=<MulBackward0>)